In [1]:
import dash
# import dash_auth
# from users import USERNAME_PASSWORD_PAIRS
from dash import dcc
from dash import html
import plotly
import dash_bootstrap_components as dbc
# import dash_daq as dq
from jupyter_dash import JupyterDash
from dash import Dash
import pandas as pd  
import joblib
import base64, io, os
from dash import  dash_table
import dash
import pandas as pd
from dash import dash_table
import pickle
from dash.dependencies import Input, Output, State, ALL, MATCH
import json
# from functools import reduce
from dash import Input, Output, State, html
from tqdm.notebook import trange, tqdm
import pickle
import numpy as np
import pandas as pd
# from cryptography.fernet import Fernet
import json
import io, base64, os
import pandas as pd
import datetime
import time
import re
import plotly.express as px
import plotly.graph_objects as go
from dash_canvas.utils import array_to_data_url

import geopandas
import tifffile as tiff
import cv2
import matplotlib; matplotlib.rcParams['figure.dpi']=300
import matplotlib.pyplot as plt
import pickle
import numpy as np
from pathpretrain.utils import generate_tissue_mask
import scipy.ndimage as ndimage
from skimage.util import compare_images
from skimage.measure import regionprops_table
from skimage import exposure
import tempfile
import matplotlib.image as mpimg
from skimage import data, draw
from io import BytesIO
import copy
# from scipy import ndimage

In [2]:
with open("./data/match_dict.pkl", "rb") as file:
    match_dict = pickle.load(file)

annotation_data = geopandas.read_file("./data/_SS12251_140745.geojson")
im_large = tiff.imread("./data/_SS12251_140745.svs")
im_medium = cv2.resize(im_large, None, fx=1/7, fy=1/7)
im_small = cv2.resize(im_large, None, fx=1/15, fy=1/15)

with open("./data/011320_002_N_HR_dict.pkl", "rb") as input_file:
    metal_data = pickle.load(input_file)

In [3]:
def crop_slide(image_array):
#     slide_file = list(filter(lambda x: match_dict[x] == imageId, match_dict))[0]    
    im_large=image_array#tifffile.imread(f"/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/metals/breast/new_slide_images_4_6_23/{slide_file}.svs")

    tissue_mask=generate_tissue_mask(im_large*5,compression=32.,threshold=240,kernel=10,gray_before_close=True) 
    compression=32
    
    # Metal image is only portion of slide
#     if (imageId == "092619_007_T_HER2"):    
        
#     if (imageId == "102720_003_T_TNBC"):
#         labels_small=ndimage.label(cv2.resize(tissue_mask.astype(np.uint8),None,fx=1/compression,fy=1/compression,interpolation=cv2.INTER_NEAREST))[0]
#         lbl_table=pd.DataFrame(regionprops_table(labels_small,properties=['bbox','area',"label"])).sort_values("area",ascending=False)
#         xmin,ymin,xmax,ymax=(lbl_table.iloc[1][[col for col in lbl_table if "bbox" in col]]*compression).astype(int)   
#     else:  
    labels_small=ndimage.label(cv2.resize(tissue_mask.astype(np.uint8),None,fx=1/compression,fy=1/compression,interpolation=cv2.INTER_NEAREST))[0]
    lbl_table=pd.DataFrame(regionprops_table(labels_small,properties=['bbox','area',"label"])).sort_values("area",ascending=False)
    xmin,ymin,xmax,ymax=(lbl_table.iloc[0][[col for col in lbl_table if "bbox" in col]]*compression).astype(int)   
    
    im_large_crop=im_large[xmin:xmax,ymin:ymax].copy()
    im_large_crop[~tissue_mask[xmin:xmax,ymin:ymax]]=255
    
    return im_large_crop, [[xmin, ymin], [xmax, ymax]], tissue_mask

In [4]:
im_large_crop, minmax_list, tissue_mask = crop_slide(im_large)
im_small_crop = cv2.resize(im_large_crop, None, fx=1/15, fy=1/15)

hne_image = px.imshow(im_small_crop)
metal_image = px.imshow(np.log(metal_data['metals']['Ca44'])+10, color_continuous_scale='jet')

/Users/luyunrui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning:

divide by zero encountered in log

/Users/luyunrui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning:

invalid value encountered in log



In [5]:
annotation_data['annot']=annotation_data['classification'].map(lambda x: x.get("name","none"))

gp2=annotation_data.copy()
gp2['geometry']=gp2['geometry'].scale(1/7,1/7,origin=(0,0))
gp2['coords'] = gp2['geometry'].apply(lambda x: x.representative_point().coords[:])
gp2['coords'] = [coords[0] for coords in gp2['coords']]

In [61]:
app = JupyterDash(__name__, 
                  external_stylesheets=[dbc.themes.SPACELAB], 
                 )

markdown_text_title = '''
# **Metals Demo**
'''

all_relayout_data = {'shapes': [], 'dragmode': ''}

type_to_color_dict = {'immune': '#6C4516', 'tumor': '#1616A7'}
color_to_type_dict = {'#6C4516': 'immune', '#1616A7': 'tumor'}

tab_Data_content = dbc.Card(
    dbc.CardBody(
        [
            html.P("This is Data tab", className="card-text"),
        ]
    ),
    className="mt-3",
)

image_no_axis_layout = go.Layout(
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
)

hne_image.update_layout(image_no_axis_layout)
metal_image.update_layout(image_no_axis_layout)
hne_image.update_layout(title=dict(text="H&E Image", x=0.5, y=0.95, xanchor='center'))

def path_to_indices(path, scale_factor):
    """From SVG path to numpy array of coordinates, each row being a (row, col) point
    """
    indices_str = [
        el.replace("M", "").replace("Z", "").split(",") for el in path.split("L")
    ]
    return np.rint(np.array(indices_str, dtype=float)*(1/scale_factor)).astype(np.int)

def path_to_mask(path, shape, scale_factor, padded_rows, padded_columns):
    path_work = path_to_indices(path, scale_factor)
    path_work[:, 0][path_work[:, 0] == padded_columns] = (padded_columns-1)
    path_work[:, 1][path_work[:, 1] == padded_rows] = (padded_rows-1)
    cols, rows = path_work.T
    rr, cc = draw.polygon(rows, cols)
    mask = np.zeros(shape, dtype=np.bool)
    mask[rr, cc] = True
    mask = ndimage.binary_fill_holes(mask)
    return mask

def update_metal_image(selected_metal, 
                       np_log, 
                      ):
    if np_log == 'With np log transform':
        original_metal_image = metal_data['metals'][selected_metal]
        cmap_jet = plt.cm.get_cmap('jet')
        original_metal_image[original_metal_image <= 0] = 0.000001
        original_metal_image = np.nan_to_num(original_metal_image, nan=0.000001)
        original_metal_image = np.log(original_metal_image)+10
        original_metal_image_normalized = (original_metal_image - original_metal_image.min()) / (original_metal_image.max() - original_metal_image.min())
        metal_image_data = cmap_jet(original_metal_image_normalized)
        metal_image_data = metal_image_data[:, :, :3]
        metal_image = px.imshow(metal_image_data)
    else:
        metal_image_data = metal_data['metals'][selected_metal]
        metal_image = px.imshow(metal_image_data, color_continuous_scale='jet')
        metal_image.layout.coloraxis.showscale = False
    metal_image.update_layout(image_no_axis_layout)
    metal_image.update_layout(title=dict(text="Metal Image - {}".format(selected_metal), x=0.5, y=0.95, xanchor='center'))
    return metal_image

tab_Visualization_content = dbc.Card(
    dbc.CardBody(
        [
            dbc.Row(
            [
                dbc.Col(dcc.Graph(id="hne_image", figure=hne_image,), 
                        width=6),
                dbc.Col(dcc.Graph(id="metal_image", figure=metal_image,),
                        width=6),
            ]
        ),
            dbc.Row([
                dcc.Dropdown(
                            id='metal_dropdown',
                            options=list(metal_data['metals'].keys()),
                            value='Ca44',
                            searchable=True, 
                            style={'width': '110px'},
                                ),
                dcc.Dropdown(
                            id='np_log_dropdown',
                            options=['With np log transform', 'Without np log transform'],
                            value='With np log transform',
                            style={'width': '300px'},
                                ),
            ], justify="end",),
        ]
    ),
    className="mt-3",
)

tab_Coregister_content = dbc.Card(
    dbc.CardBody(
        [
            html.P("This is Coregister tab", className="card-text"),
        ]
    ),
    className="mt-3",
)

original_image = px.imshow(im_medium)
original_image.update_layout(image_no_axis_layout)
original_image.update_layout(title=dict(text="H&E Image", x=0.5, y=0.95, xanchor='center'))
original_image.update_layout(margin={'l': 0, 'r': 0, 't': 0, 'b': 0})


padded_rows = 1000
padded_columns = 1600

original_metal_image = metal_data['metals']['Ca44']
rows_padding = max(0, padded_rows - original_metal_image.shape[0])
columns_padding = max(0, padded_columns - original_metal_image.shape[1])
top_pad = rows_padding // 2
bottom_pad = rows_padding - top_pad
left_pad = columns_padding // 2
right_pad = columns_padding - left_pad
padded_metal_image = np.pad(original_metal_image, ((top_pad, bottom_pad), (left_pad, right_pad)), mode='constant')

cmap_jet = plt.cm.get_cmap('jet')
padded_metal_image[padded_metal_image <= 0] = 0.000001
padded_metal_image = np.nan_to_num(padded_metal_image, nan=0.000001)
padded_metal_image = np.log(padded_metal_image)+10
padded_metal_image_normalized = (padded_metal_image - padded_metal_image.min()) / (padded_metal_image.max() - padded_metal_image.min())
padded_metal_image_rgb = cmap_jet(padded_metal_image_normalized)
padded_metal_image_rgb = padded_metal_image_rgb[:, :, :3]

blank_figure = go.Figure()

blank_img_width = padded_columns
blank_img_height = padded_rows
scale_factor = 0.35

blank_figure.add_trace(
    go.Scatter(
        x=[0, blank_img_width * scale_factor],
        y=[0, blank_img_height * scale_factor],
        mode="markers",
        marker_opacity=0
    )
)

blank_figure.update_xaxes(
    visible=False,
    range=[0, blank_img_width * scale_factor]
)

blank_figure.update_yaxes(
    visible=False,
    range=[0, blank_img_height * scale_factor],
    scaleanchor="x",
)
    
blank_figure.add_layout_image(
    dict(
        x=0,
        sizex=blank_img_width * scale_factor,
        y=blank_img_height * scale_factor,
        sizey=blank_img_height * scale_factor,
        xref="x",
        yref="y",
        opacity=1.0,
        layer="below",
        sizing="stretch",
        source=array_to_data_url((padded_metal_image_rgb * 255).astype(np.uint8)),)
)

blank_figure.update_layout(
    width=blank_img_width * scale_factor,
    height=blank_img_height * scale_factor,
    margin={"l": 0, "r": 0, "t": 0, "b": 0},
)

image_annotation_card = dbc.Card(
    children = [dbc.CardHeader(html.H2("Annotation area")),
        dbc.CardBody(
            [
                dcc.Graph(
                    id="annotate_metal_image",
                    figure=blank_figure,
                    config={"modeBarButtonsToAdd": ["drawrect", "eraseshape", "drawclosedpath",], 
                            'modeBarButtonsToRemove': ['zoom2d', 'pan2d', 'zoomIn2d', 'zoomOut2d']},
                ),
                dcc.Store(id='annotation_store', data=[]),
            ]
        ),
        dbc.CardFooter(
            [
                dcc.Markdown(
                    "To annotate the above image"
                    "rectangle .\n\n"
                    "**Choose metals**:"
                ),
                dbc.Row([
                    dcc.Dropdown(
                                id='type_dropdown_annotation',
                                options=['immune', 'tumor'],
                                value='immune',
                                searchable=True, 
                                style={'width': '110px'},
                                    ),
                    dcc.Dropdown(
                                id='metal_dropdown_annotation',
                                options=list(metal_data['metals'].keys()),
                                value='Ca44',
                                searchable=True, 
                                style={'width': '110px'},
                                    ),
                ], justify="center",),
                
#                 dbc.ButtonGroup(
#                     [
#                         dbc.Button("Previous image", id="previous", outline=True),
#                         dbc.Button("Next image", id="next", outline=True),
#                     ],
#                     size="lg",
#                     style={"width": "100%"},
#                 ),
            ]
        ),]
)

result_df = pd.DataFrame({'type': [], 'metal': [], 
                              'mean': [], 'std': []})
annotation_result_table = dbc.Table.from_dataframe(result_df, striped=True, bordered=True, hover=True)

annotated_data_card = dbc.Card(
    children = [dbc.CardHeader(html.H2("Table")),
        dbc.CardBody(
            [
#                 html.P("This is Coregister tab", className="card-text"),
#                 annotation_result_table, 
                html.Div(id='table_container', children = annotation_result_table),
            ]
        ),
        dbc.CardFooter(
            [
                dbc.Button('Calculate Result', id='update_table_button', size="sm", n_clicks=0, className="me-1"),
#                 dbc.Button("Large button", size="lg", className="me-1"),
#                 dcc.Markdown(
#                     "To annotate the above image"
#                     "rectangle .\n\n"
#                     "**Choose metals**:"
#                 ),
                
            ]
        ),]
)

image_annotation_table = dbc.Row(
                                [
                                    dbc.Col(dcc.Graph(figure=original_image), width=4),
                                    dbc.Col(image_annotation_card, width=5),
                                    dbc.Col(annotated_data_card, md=3),
                                    ], 
justify="start",)

tab_Measure_content = dbc.Card(
    dbc.CardBody(
        [
            image_annotation_table,
        ]
    ),
    className="mt-3",
)

all_tabs = dbc.Tabs(
    [
        dbc.Tab(tab_Data_content, label="Data"),
        dbc.Tab(tab_Visualization_content, label="Visualization"),
        dbc.Tab(tab_Coregister_content, label="Co-register"),
        dbc.Tab(tab_Measure_content, label="Measure"),
    ]
)

app.layout = html.Div([dcc.Markdown(children=markdown_text_title),
                       all_tabs])

@app.callback(
    Output('metal_image', 'figure'),
    [Input('metal_dropdown', 'value'), 
     Input('np_log_dropdown', 'value')]
)

def update_metal_image_callback(selected_metal, np_log):
    return update_metal_image(selected_metal, np_log)

@app.callback(
    Output('annotate_metal_image', 'figure'),
    Input('metal_dropdown_annotation', 'value'),
    Input('annotate_metal_image', 'relayoutData'),
    Input('type_dropdown_annotation', 'value'),
)

def update_annotation_callback(selected_metal, 
                               relayout_data,
                               annotation_type,
                              ):
    padded_rows = 1000
    padded_columns = 1600

    original_metal_image = metal_data['metals'][selected_metal]
    rows_padding = max(0, padded_rows - original_metal_image.shape[0])
    columns_padding = max(0, padded_columns - original_metal_image.shape[1])
    top_pad = rows_padding // 2
    bottom_pad = rows_padding - top_pad
    left_pad = columns_padding // 2
    right_pad = columns_padding - left_pad
    padded_metal_image = np.pad(original_metal_image, ((top_pad, bottom_pad), (left_pad, right_pad)), mode='constant')

    cmap_jet = plt.cm.get_cmap('jet')
    padded_metal_image[padded_metal_image <= 0] = 0.000001
    padded_metal_image = np.nan_to_num(padded_metal_image, nan=0.000001)
    padded_metal_image = np.log(padded_metal_image)+10
    padded_metal_image_normalized = (padded_metal_image - padded_metal_image.min()) / (padded_metal_image.max() - padded_metal_image.min())
    padded_metal_image_rgb = cmap_jet(padded_metal_image_normalized)
    padded_metal_image_rgb = padded_metal_image_rgb[:, :, :3]

    blank_figure = go.Figure()

    blank_img_width = padded_columns
    blank_img_height = padded_rows
    scale_factor = 0.35

    blank_figure.add_trace(
        go.Scatter(
            x=[0, blank_img_width * scale_factor],
            y=[0, blank_img_height * scale_factor],
            mode="markers",
            marker_opacity=0
        )
    )

    blank_figure.update_xaxes(
        visible=False,
        range=[0, blank_img_width * scale_factor]
    )

    blank_figure.update_yaxes(
        visible=False,
        range=[0, blank_img_height * scale_factor],
        scaleanchor="x",
    )

    blank_figure.add_layout_image(
        dict(
            x=0,
            sizex=blank_img_width * scale_factor,
            y=blank_img_height * scale_factor,
            sizey=blank_img_height * scale_factor,
            xref="x",
            yref="y",
            opacity=1.0,
            layer="below",
            sizing="stretch",
            source=array_to_data_url((padded_metal_image_rgb * 255).astype(np.uint8)),)
    )
    
    blank_figure.update_layout(
        width=blank_img_width * scale_factor,
        height=blank_img_height * scale_factor,
        margin={"l": 0, "r": 0, "t": 0, "b": 0},
    )
    
#     print('relayout_data:', relayout_data)
#     print('all_relayout_data:', all_relayout_data)
    
    if 'shapes' in list(relayout_data.keys()):
        if len(relayout_data['shapes']) > len(all_relayout_data['shapes']):
            append_relayout = relayout_data['shapes'][-1]
            append_relayout['line']['color'] = type_to_color_dict[annotation_type]
            all_relayout_data['shapes'].append(append_relayout)
        elif len(relayout_data['shapes']) < len(all_relayout_data['shapes']):
#             all_relayout_data = all_relayout_data
#         else:
            all_relayout_data['shapes'] = relayout_data['shapes']
#         for one_shape in all_relayout_data['shapes']:
#             one_shape['line']['width'] = 2

    if 'dragmode' in list(relayout_data.keys()):
        all_relayout_data['dragmode'] = relayout_data['dragmode']
        
#     print('relayout_data:', relayout_data)
#     print('all_relayout_data:', all_relayout_data)
    for one_shape in all_relayout_data['shapes']:
            one_shape['line']['width'] = 2
    blank_figure['layout'].update(all_relayout_data)
    
#     print('relayout_data_1:', relayout_data)
#     print('all_relayout_data_1:', all_relayout_data)

    return blank_figure


# @app.callback(
#     Output('table_container', 'children'),
#     Input('annotate_metal_image', 'relayoutData'),
# )

# def update_annotation_callback(relayout_data,):
#     padded_rows = 1000
#     padded_columns = 1600
#     scale_factor = 0.35
    
#     all_type_list = []
#     for one_shape in all_relayout_data['shapes']:
#         all_type_list.append(color_to_type_dict[one_shape['line']['color']])
#     all_type_list = list(set(all_type_list))
    
#     all_type_area_list_dict = {}
#     for one_type in all_type_list:
#         all_type_area_list_dict[one_type] = []
    
#     for one_shape in all_relayout_data['shapes']:
#         all_type_area_list_dict[color_to_type_dict[one_shape['line']['color']]].append(one_shape['path'])
        
#     all_type_area_dict = {}
# #     for one_type in all_type_list:
# #         all_type_area_dict[one_type] = []
        
#     for one_type in list(all_type_area_list_dict.keys()):
#         for one_path in all_type_area_list_dict[one_type]:
#             if one_type not in list(all_type_area_dict.keys()):
#                 all_type_area_dict[one_type] = path_to_mask(one_path, (padded_rows, padded_columns), 
#                                                             scale_factor, 
#                                                             padded_rows, padded_columns)
#             else:
#                 all_type_area_dict[one_type] += path_to_mask(one_path, (padded_rows, padded_columns), 
#                                                             scale_factor, 
#                                                             padded_rows, padded_columns)
    
# #     df_column_list = ['type', 'mean', 'std']
#     type_df_list = []
#     metal_df_list = []
#     mean_df_list = []
#     std_df_list = []
    
#     for one_type in list(all_type_area_dict.keys()):
#         one_area = all_type_area_dict[one_type]
#         for one_metal in list(metal_data['metals'].keys()):
#             original_metal_image = metal_data['metals'][one_metal]
#             rows_padding = max(0, padded_rows - original_metal_image.shape[0])
#             columns_padding = max(0, padded_columns - original_metal_image.shape[1])
#             top_pad = rows_padding // 2
#             bottom_pad = rows_padding - top_pad
#             left_pad = columns_padding // 2
#             right_pad = columns_padding - left_pad
#             padded_metal_image = np.pad(original_metal_image, 
#                                         ((top_pad, bottom_pad), 
#                                          (left_pad, right_pad)), 
#                                         mode='constant')
#             padded_metal_image = np.nan_to_num(padded_metal_image, nan=0.000001)
            
#             type_df_list.append(one_type)
#             metal_df_list.append(one_metal)
#             mean_df_list.append(round(padded_metal_image[one_area].mean(), 3))
#             std_df_list.append(round(padded_metal_image[one_area].std(), 3))
            
#     result_df = pd.DataFrame({'type': type_df_list, 'metal': metal_df_list, 
#                               'mean': mean_df_list, 'std': std_df_list})
    
# #     padded_rows = 1000
# #     padded_columns = 1600
    
# #     original_metal_image = metal_data['metals'][selected_metal]
# #     rows_padding = max(0, padded_rows - original_metal_image.shape[0])
# #     columns_padding = max(0, padded_columns - original_metal_image.shape[1])
# #     top_pad = rows_padding // 2
# #     bottom_pad = rows_padding - top_pad
# #     left_pad = columns_padding // 2
# #     right_pad = columns_padding - left_pad
# #     padded_metal_image = np.pad(original_metal_image, 
# #                                 ((top_pad, bottom_pad), 
# #                                  (left_pad, right_pad)), 
# #                                 mode='constant')

# #     all_relayout_data
    
#     annotation_result_table = dbc.Table.from_dataframe(result_df, striped=True, bordered=True, hover=True)
    
#     return annotation_result_table


@app.callback(
    Output('table_container', 'children'),
    Input('update_table_button', 'n_clicks'),
    State('table_container', 'children'),
)

def update_table_callback(n_clicks, table_children):
    if n_clicks is None:
        return table_children
    
    if n_clicks == 0:
        return table_children

#     # Perform your calculations here. For this example, we'll add 1000 to the 'Salary' column.
#     print(n_clicks)
#     print('-------')
#     updated_data = data.copy()
#     updated_data['Salary'] += n_clicks * 1000
#     print(updated_data)
#     print('-------')
    
#     annotation_result_table = dbc.Table.from_dataframe(updated_data, 
#                                                    striped=True, bordered=True, hover=True)
#     print('ss')

#     return annotation_result_table
    
    padded_rows = 1000
    padded_columns = 1600
    scale_factor = 0.35
    
    all_type_list = []
    for one_shape in all_relayout_data['shapes']:
        all_type_list.append(color_to_type_dict[one_shape['line']['color']])
    all_type_list = list(set(all_type_list))
    
    all_type_area_list_dict = {}
    for one_type in all_type_list:
        all_type_area_list_dict[one_type] = []
    
    for one_shape in all_relayout_data['shapes']:
        all_type_area_list_dict[color_to_type_dict[one_shape['line']['color']]].append(one_shape['path'])
        
    all_type_area_dict = {}
#     for one_type in all_type_list:
#         all_type_area_dict[one_type] = []
        
    for one_type in list(all_type_area_list_dict.keys()):
        for one_path in all_type_area_list_dict[one_type]:
            if one_type not in list(all_type_area_dict.keys()):
                all_type_area_dict[one_type] = path_to_mask(one_path, (padded_rows, padded_columns), 
                                                            scale_factor, 
                                                            padded_rows, padded_columns)
            else:
                all_type_area_dict[one_type] += path_to_mask(one_path, (padded_rows, padded_columns), 
                                                            scale_factor, 
                                                            padded_rows, padded_columns)
    
#     df_column_list = ['type', 'mean', 'std']
    type_df_list = []
    metal_df_list = []
    mean_df_list = []
    std_df_list = []
    
    for one_type in list(all_type_area_dict.keys()):
        one_area = all_type_area_dict[one_type]
        for one_metal in list(metal_data['metals'].keys()):
            original_metal_image = metal_data['metals'][one_metal]
            rows_padding = max(0, padded_rows - original_metal_image.shape[0])
            columns_padding = max(0, padded_columns - original_metal_image.shape[1])
            top_pad = rows_padding // 2
            bottom_pad = rows_padding - top_pad
            left_pad = columns_padding // 2
            right_pad = columns_padding - left_pad
            padded_metal_image = np.pad(original_metal_image, 
                                        ((top_pad, bottom_pad), 
                                         (left_pad, right_pad)), 
                                        mode='constant')
            padded_metal_image = np.nan_to_num(padded_metal_image, nan=0.000001)
            
            type_df_list.append(one_type)
            metal_df_list.append(one_metal)
            mean_df_list.append(round(padded_metal_image[one_area].mean(), 3))
            std_df_list.append(round(padded_metal_image[one_area].std(), 3))
            
    result_df = pd.DataFrame({'type': type_df_list, 'metal': metal_df_list, 
                              'mean': mean_df_list, 'std': std_df_list})
    
#     padded_rows = 1000
#     padded_columns = 1600
    
#     original_metal_image = metal_data['metals'][selected_metal]
#     rows_padding = max(0, padded_rows - original_metal_image.shape[0])
#     columns_padding = max(0, padded_columns - original_metal_image.shape[1])
#     top_pad = rows_padding // 2
#     bottom_pad = rows_padding - top_pad
#     left_pad = columns_padding // 2
#     right_pad = columns_padding - left_pad
#     padded_metal_image = np.pad(original_metal_image, 
#                                 ((top_pad, bottom_pad), 
#                                  (left_pad, right_pad)), 
#                                 mode='constant')

#     all_relayout_data
    
    annotation_result_table = dbc.Table.from_dataframe(result_df, striped=True, bordered=True, hover=True)
    
    return annotation_result_table
    
    


In [62]:
app.run_server(debug=False,
               mode="external",
               host='localhost', 
              )

 * Running on http://localhost:8050 (Press CTRL+C to quit)
127.0.0.1 - - [01/Aug/2023 14:14:40] "GET /_alive_e5bebdda-664b-4806-9f3a-330fe17b280e HTTP/1.1" 200 -


Dash app running on http://localhost:8050/


127.0.0.1 - - [01/Aug/2023 14:14:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2023 14:14:47] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2023 14:14:48] "GET /_favicon.ico?v=2.5.0 HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2023 14:14:49] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2023 14:14:50] "GET /_dash-component-suites/dash/dcc/async-markdown.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Aug/2023 14:14:50] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Aug/2023 14:14:50] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Aug/2023 14:14:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2023 14:14:50] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Aug/2023 14:14:50] "GET /_dash-component-suites/dash/dcc/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Aug/2023 14:14:51] "POST /_dash-update-component HTTP/1.1" 200 -
1

In [60]:
app = JupyterDash(__name__, 
                  external_stylesheets=[dbc.themes.SPACELAB], 
                 )

# Sample data for demonstration purposes
data = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie', 'David'],
    'Age': [25, 30, 22, 28],
    'Salary': [50000, 60000, 45000, 55000]
})

annotation_result_table = dbc.Table.from_dataframe(data, 
                                                   striped=True, bordered=True, hover=True)

# Layout of the app
app.layout = html.Div([
    html.Button('Update Table', id='update-button', n_clicks=0),
#     dbc.Table.from_dataframe(id = 'show_table', children = data, striped=True, bordered=True, hover=True),
    html.Div(id='table_container', children = annotation_result_table),
])

# Callback to update the table when the button is clicked
@app.callback(
    Output('table_container', 'children'),
    Input('update-button', 'n_clicks'),
    State('table_container', 'children'),
#     State('table', 'data'),
)
def update_table(n_clicks, table_children):
    if n_clicks is None:
        return table_children  # Return the initial data when the app first loads
    
    if n_clicks == 0:
        return table_children  # Return the initial data when the app first loads

    # Perform your calculations here. For this example, we'll add 1000 to the 'Salary' column.
    print(n_clicks)
    print('-------')
    updated_data = data.copy()
    updated_data['Salary'] += n_clicks * 1000
    print(updated_data)
    print('-------')
    
    annotation_result_table = dbc.Table.from_dataframe(updated_data, 
                                                   striped=True, bordered=True, hover=True)
    print('ss')

    return annotation_result_table


app.run_server(debug=False,
               mode="external",
               host='localhost', 
              )


 * Running on http://localhost:8050 (Press CTRL+C to quit)
127.0.0.1 - - [01/Aug/2023 14:09:43] "GET /_alive_e5bebdda-664b-4806-9f3a-330fe17b280e HTTP/1.1" 200 -


Dash app running on http://localhost:8050/


127.0.0.1 - - [01/Aug/2023 14:09:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2023 14:09:46] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2023 14:09:46] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2023 14:09:46] "GET /_favicon.ico?v=2.5.0 HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2023 14:09:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2023 14:09:46] "POST /_dash-update-component HTTP/1.1" 200 -


1
-------
      Name  Age  Salary
0    Alice   25   51000
1      Bob   30   61000
2  Charlie   22   46000
3    David   28   56000
-------
ss


127.0.0.1 - - [01/Aug/2023 14:09:47] "POST /_dash-update-component HTTP/1.1" 200 -


...

4
-------
      Name  Age  Salary
0    Alice   25   54000
1      Bob   30   64000
2  Charlie   22   49000
3    David   28   59000
-------
ss
5
-------
      Name  Age  Salary
0    Alice   25   55000
1      Bob   30   65000
2  Charlie   22   50000
3    David   28   60000
-------
ss


127.0.0.1 - - [01/Aug/2023 14:09:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2023 14:09:48] "POST /_dash-update-component HTTP/1.1" 200 -


6
-------
      Name  Age  Salary
0    Alice   25   56000
1      Bob   30   66000
2  Charlie   22   51000
3    David   28   61000
-------
ss
7
-------
      Name  Age  Salary
0    Alice   25   57000
1      Bob   30   67000
2  Charlie   22   52000
3    David   28   62000
-------
ss


127.0.0.1 - - [01/Aug/2023 14:09:48] "POST /_dash-update-component HTTP/1.1" 200 -


8
-------
      Name  Age  Salary
0    Alice   25   58000
1      Bob   30   68000
2  Charlie   22   53000
3    David   28   63000
-------
ss


In [48]:
data = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie', 'David'],
    'Age': [25, 30, 22, 28],
    'Salary': [50000, 60000, 45000, 55000]
})

In [49]:
data

,Name,Age,Salary
0,Alice,25,50000
1,Bob,30,60000
2,Charlie,22,45000
3,David,28,55000


In [50]:
data['Salary'] += 1000

In [51]:
data

,Name,Age,Salary
0,Alice,25,51000
1,Bob,30,61000
2,Charlie,22,46000
3,David,28,56000


In [10]:
test_1 = np.array([[True, False, False], 
                 [False, True, False], 
                 [False, False, False]])
test_2 = np.array([[False, False, False], 
                 [False, True, True], 
                 [False, False, False]])

In [11]:
test_1 + test_2

array([[ True, False, False],
       [False,  True,  True],
       [False, False, False]])

In [12]:
test_3 = np.array([[1, 2, 3], 
                 [3, 4, 5], 
                 [5, 6, 7]])

In [15]:
test_3[test_1 + test_2].mean()

array([1, 4, 5])

In [16]:
test_3[test_1 + test_2].std()

1.699673171197595

In [10]:
px.colors.qualitative.Light24

['#FD3216',
 '#00FE35',
 '#6A76FC',
 '#FED4C4',
 '#FE00CE',
 '#0DF9FF',
 '#F6F926',
 '#FF9616',
 '#479B55',
 '#EEA6FB',
 '#DC587D',
 '#D626FF',
 '#6E899C',
 '#00B5F7',
 '#B68E00',
 '#C9FBE5',
 '#FF0092',
 '#22FFA7',
 '#E3EE9E',
 '#86CE00',
 '#BC7196',
 '#7E7DCD',
 '#FC6955',
 '#E48F72']

In [306]:
path_to_indices('M1226.2127472986629,747.7611332983522L1236.162996054882,867.164118372979L1290.8893642140858,976.6168546913871L1395.3669761543842,1036.3183472287005L1514.7699612290112,1046.2685959849193L1634.1729463036381,991.5422278257154L1668.9988169504043,901.9899890197452L1678.949065706623,807.4626258356656L1659.0485681941852,737.8108845421333L1624.222697547419,707.9601382734766Z')


/Users/luyunrui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



array([[1226,  748],
       [1236,  867],
       [1291,  977],
       [1395, 1036],
       [1515, 1046],
       [1634,  992],
       [1669,  902],
       [1679,  807],
       [1659,  738],
       [1624,  708]])

In [17]:
padded_metal_image_rgb.shape

(1000, 1600, 3)

In [329]:
path_to_mask('M0.66876220703125,267.1999969482422L0.66876220703125,53.19999694824219L19.66876220703125,4.1999969482421875L51.66876220703125,0L132.66876220703125,0.1999969482421875L242.66876220703125,15.199996948242188L296.66876220703125,13.199996948242188L349.66876220703125,8.199996948242188L405.66876220703125,12.199996948242188L404.66876220703125,12.199996948242188L480,5.1999969482421875L480,5.1999969482421875L480,147.1999969482422L467.66876220703125,176.1999969482422L444.66876220703125,210.1999969482422L441.66876220703125,242.1999969482422L464.66876220703125,249.1999969482422L471.66876220703125,254.1999969482422L471.66876220703125,255.1999969482422Z', 
             (900, 1600), 
            ).sum()



/Users/luyunrui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/Users/luyunrui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-

1319595

In [ ]:
# foo[foo == 0] = m

In [330]:
1319595/(900*1600)

0.9163854166666666

In [302]:
1600*0.3

480.0

In [328]:
test_path = 'M1.283762541529971,866.56260078403L83.96881592231288,863.8953409975531L81.30155613583602,789.2120669762008L59.96347784402107,781.2102876167702L-0.5,786.544807189724Z'
# def path_to_indices(path):
#     """From SVG path to numpy array of coordinates, each row being a (row, col) point
#     """
#     indices_str = [
#         el.replace("M", "").replace("Z", "").split(",") for el in path.split("L")
#     ]
#     return np.rint(np.array(indices_str, dtype=float)).astype(np.int)
def path_to_indices(path):
    """From SVG path to numpy array of coordinates, each row being a (row, col) point
    """
    indices_str = [
        el.replace("M", "").replace("Z", "").split(",") for el in path.split("L")
    ]
    return np.rint(np.array(indices_str, dtype=float)*(10/3)).astype(np.int)
def path_to_mask(path, shape):
    """From SVG path to a boolean array where all pixels enclosed by the path
    are True, and the other pixels are False.
    """
    path_work = path_to_indices(path)
    path_work[path_work == 1600] = 1599
    path_work[path_work == 900] = 899
    cols, rows = path_work.T
#     foo[foo == 0] = m
    rr, cc = draw.polygon(rows, cols)
    mask = np.zeros(shape, dtype=np.bool)
    mask[rr, cc] = True
    mask = ndimage.binary_fill_holes(mask)
    return mask

In [58]:
path_to_indices(test_path)

/Users/luyunrui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



array([[  1, 867],
       [ 84, 864],
       [ 81, 789],
       [ 60, 781],
       [  0, 787]])

In [59]:
path_to_mask(test_path, metal_data['metals']['Ca44'].shape)

/Users/luyunrui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/Users/luyunrui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-n

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])